In [295]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import matplotlib.pyplot as plt
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import svm


In [296]:
# settings
np.random.seed(2020)
name = "Dblp"
d = 128
come_model_type = "BGMM"
ks = [2, 5, 10, 20]

In [297]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [298]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/anton/minico

k = 2
k = 5
k = 10
k = 20


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.447244,-0.590060,-0.327605,0.759601,0.407560,0.058691,0.100031,0.519430,0.105071,0.309200,...,0.026815,-0.170982,-0.574228,-0.198281,-0.451050,0.011072,-0.094810,-0.301875,1,0
1,0.295918,-0.558974,-0.200630,0.670498,0.255454,0.012444,0.046877,0.438116,-0.062488,0.382698,...,-0.077961,-0.241738,-0.520454,-0.098120,-0.499365,-0.048436,0.035603,-0.250949,1,2
2,0.533605,-0.535543,-0.390154,0.778972,0.445067,0.030342,0.050596,0.546266,0.181827,0.356746,...,0.023890,-0.090602,-0.596374,-0.249288,-0.443732,-0.064824,-0.046458,-0.358123,1,2
3,0.422736,-0.478823,-0.391336,0.637420,0.339128,-0.047656,0.112250,0.464919,0.160469,0.317239,...,-0.017747,-0.144513,-0.465714,-0.285970,-0.305572,0.000968,-0.058966,-0.357066,1,0
4,0.325676,-0.428440,-0.432361,0.703664,0.361787,0.000723,-0.070736,0.472260,0.272814,0.312631,...,0.039992,-0.145147,-0.551694,-0.296413,-0.290567,-0.063562,-0.169452,-0.348867,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.257980,-0.395774,-0.185478,0.552952,0.410871,-0.034553,-0.067511,0.413107,0.183474,0.325979,...,-0.171239,-0.060339,-0.347124,-0.287109,-0.128301,0.030241,0.094815,-0.255052,1,0
13180,-0.020099,-0.540597,-0.269626,0.003425,0.431760,0.091559,0.119777,0.100602,0.358634,0.445394,...,-0.318657,0.178803,-0.034317,0.113252,0.093031,-0.522664,0.109797,0.340559,0,3
13181,-0.118595,-0.484969,-0.095687,0.009911,0.409799,-0.006928,0.062702,0.008954,0.505634,0.593392,...,-0.070694,0.259097,-0.084647,0.026951,-0.060800,-0.500006,0.138947,0.363550,0,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.070362,-0.349749,-0.039339,0.317448,0.314261,-0.113492,0.198119,0.249685,0.185212,0.415116,...,-0.116847,0.216113,-0.288452,-0.003144,-0.033984,-0.015064,0.099052,0.109553,0.386074,2.190382
std,0.158703,0.187695,0.180287,0.210781,0.165113,0.194175,0.185949,0.172968,0.186557,0.152076,...,0.149657,0.157409,0.220734,0.148219,0.179455,0.163438,0.173225,0.249231,0.486866,1.315339
min,-0.508291,-1.023050,-0.624764,-0.434922,-0.292775,-0.728536,-0.466597,-0.464700,-0.451996,-0.187440,...,-0.741044,-0.457601,-0.949299,-0.708024,-0.760009,-0.728847,-0.512301,-0.561539,0.000000,0.000000
25%,-0.032193,-0.487592,-0.161421,0.168124,0.208116,-0.263246,0.068560,0.132015,0.043462,0.317043,...,-0.216404,0.111103,-0.451154,-0.091365,-0.161933,-0.116730,-0.018078,-0.096275,0.000000,1.000000
50%,0.071219,-0.348648,-0.047358,0.329033,0.321037,-0.114689,0.197320,0.254364,0.171024,0.419377,...,-0.114634,0.219112,-0.293272,-0.000578,-0.035395,-0.010756,0.092400,0.131962,0.000000,3.000000
75%,0.174777,-0.212791,0.073073,0.479097,0.422816,0.028225,0.326423,0.364569,0.318899,0.515942,...,-0.016848,0.319095,-0.113045,0.089160,0.090639,0.094135,0.211754,0.305395,1.000000,3.000000
max,0.637020,0.296944,0.650146,0.993297,0.928177,0.465188,0.867931,0.845109,0.869033,0.919915,...,0.413959,0.770119,0.419585,0.654550,0.548400,0.575931,0.699623,0.888070,1.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.434226,-0.387976,-0.342012,0.727168,0.339455,0.211082,-0.027637,0.511111,0.133573,0.330798,...,0.135571,-0.293498,-0.539234,-0.265420,-0.379549,-0.110447,-0.076284,-0.343553,2,0
1,0.265924,-0.362547,-0.275657,0.634993,0.248291,0.214578,-0.060368,0.525814,-0.084931,0.387744,...,0.069670,-0.297604,-0.484533,-0.044852,-0.506817,-0.237709,0.076055,-0.296019,2,2
2,0.539574,-0.342830,-0.470852,0.709976,0.387971,0.104439,-0.028320,0.518842,0.274218,0.313852,...,0.095333,-0.179778,-0.516522,-0.338626,-0.330501,-0.224829,-0.066941,-0.375785,2,2
3,0.404247,-0.354206,-0.359028,0.522853,0.318014,0.021533,0.011222,0.436997,0.208020,0.299709,...,0.097435,-0.197099,-0.393147,-0.290320,-0.291386,-0.063158,-0.024950,-0.379730,2,0
4,0.292309,-0.274753,-0.430262,0.620953,0.283817,0.120868,-0.159357,0.449058,0.287632,0.281005,...,0.191719,-0.216171,-0.459852,-0.342050,-0.281305,-0.127021,-0.099702,-0.363577,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.309951,-0.495111,-0.300779,0.458932,0.400343,-0.025314,-0.124034,0.430032,0.282203,0.353015,...,-0.128318,-0.118762,-0.254270,-0.394024,-0.180928,-0.113551,0.027718,-0.182381,2,0
13180,-0.179346,-0.425092,-0.131352,-0.008551,0.641813,0.157232,-0.034580,0.084456,0.400029,0.587169,...,-0.193749,0.104673,-0.087115,0.047355,0.194254,-0.676473,0.227743,0.301922,1,3
13181,-0.202129,-0.270683,-0.065851,-0.086173,0.563360,0.147709,-0.030454,0.010829,0.400087,0.621894,...,-0.073927,0.141782,-0.137634,0.127659,0.144769,-0.535276,0.204934,0.395553,1,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.056907,-0.300949,-0.051575,0.263536,0.313437,-0.110308,0.186417,0.225308,0.167554,0.427627,...,-0.084393,0.188729,-0.236242,-0.005668,-0.028079,-0.034837,0.104008,0.150095,2.364912,2.190382
std,0.175450,0.198691,0.217895,0.196683,0.176901,0.237194,0.204042,0.186604,0.206949,0.172324,...,0.164956,0.177838,0.213110,0.174463,0.191729,0.172941,0.196949,0.240662,1.575417,1.315339
min,-0.599644,-0.979674,-0.761460,-0.431146,-0.313497,-0.808270,-0.532729,-0.523907,-0.471191,-0.233847,...,-0.742946,-0.508119,-0.997244,-0.809226,-0.799460,-0.847338,-0.553988,-0.544040,0.000000,0.000000
25%,-0.053715,-0.447824,-0.196762,0.134064,0.199237,-0.298915,0.049980,0.100506,0.009227,0.319663,...,-0.193386,0.070332,-0.389503,-0.109052,-0.172523,-0.144219,-0.031847,-0.031997,1.000000,1.000000
50%,0.057838,-0.298673,-0.064082,0.269872,0.311676,-0.126139,0.193772,0.221425,0.154166,0.429794,...,-0.073870,0.194962,-0.222244,-0.006461,-0.029637,-0.036875,0.092303,0.142127,3.000000,3.000000
75%,0.173343,-0.156716,0.074690,0.398755,0.424260,0.063998,0.329895,0.345121,0.315141,0.536515,...,0.027965,0.313530,-0.076865,0.100522,0.104515,0.077643,0.232525,0.335178,4.000000,3.000000
max,0.634653,0.436895,0.787509,1.007816,1.085823,0.601460,0.865242,0.923997,0.960596,1.081570,...,0.499913,0.765260,0.444356,0.799704,0.619760,0.543685,0.853692,0.921955,4.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.251907,-0.505664,-0.419118,0.632135,0.398764,0.081527,0.015804,0.457123,0.095868,0.247154,...,0.045600,-0.252722,-0.654624,-0.176457,-0.448009,-0.104064,0.086658,-0.321366,2,0
1,0.137351,-0.473816,-0.340371,0.604147,0.302837,0.078943,-0.083839,0.405024,-0.188033,0.297292,...,0.016966,-0.342794,-0.664579,-0.027594,-0.568439,-0.113296,0.203863,-0.220902,2,2
2,0.443372,-0.421117,-0.488506,0.653578,0.401298,0.019925,-0.050461,0.480615,0.168991,0.273467,...,-0.044120,-0.168165,-0.710264,-0.287452,-0.420785,-0.182571,0.063133,-0.417098,2,2
3,0.352708,-0.390328,-0.499451,0.462635,0.294689,-0.012369,-0.043344,0.410808,0.121116,0.210863,...,-0.031745,-0.173956,-0.553134,-0.282234,-0.296926,-0.058757,0.113652,-0.329610,2,0
4,0.178464,-0.461004,-0.458270,0.548599,0.393617,0.022614,-0.141256,0.385069,0.199027,0.217831,...,0.092863,-0.190637,-0.574329,-0.330060,-0.281258,-0.205195,-0.041829,-0.356693,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.243073,-0.416906,-0.207475,0.401847,0.458673,-0.109211,-0.026420,0.370243,0.170469,0.370917,...,-0.171621,-0.185722,-0.362239,-0.377194,-0.170739,-0.172486,0.233717,-0.230589,2,0
13180,0.023682,-0.494501,-0.321138,-0.007664,0.500121,0.144919,-0.028490,0.097620,0.377990,0.602959,...,-0.225334,0.056644,-0.091681,0.213036,0.188962,-0.712634,0.035605,0.210500,6,3
13181,-0.026656,-0.329028,-0.125074,-0.065591,0.449987,0.097362,-0.135192,-0.095904,0.414919,0.643605,...,-0.031689,0.223825,-0.065080,0.090597,0.096757,-0.742413,0.061428,0.220640,6,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.061659,-0.307922,-0.043280,0.267458,0.283194,-0.110963,0.196609,0.206217,0.145598,0.401207,...,-0.110472,0.193815,-0.241077,-0.014523,-0.037759,-0.031891,0.106252,0.145180,3.841475,2.190382
std,0.186969,0.209476,0.219645,0.197165,0.191440,0.225223,0.221268,0.193866,0.211477,0.171895,...,0.179027,0.190167,0.223921,0.177488,0.198760,0.181058,0.209146,0.235364,2.416138,1.315339
min,-0.631389,-1.119389,-0.755941,-0.432110,-0.407617,-0.776992,-0.611021,-0.556540,-0.486640,-0.260419,...,-0.858273,-0.562371,-1.125345,-0.854110,-0.774439,-0.926401,-0.607830,-0.640272,0.000000,0.000000
25%,-0.057179,-0.462334,-0.194877,0.135370,0.160535,-0.278247,0.049806,0.076804,-0.010581,0.286197,...,-0.226087,0.065487,-0.395653,-0.124619,-0.181085,-0.146498,-0.034962,-0.023297,2.000000,1.000000
50%,0.057520,-0.307663,-0.052052,0.278044,0.283510,-0.124043,0.207419,0.201023,0.127342,0.404695,...,-0.097037,0.196316,-0.230280,-0.011000,-0.044257,-0.033785,0.099603,0.150573,4.000000,3.000000
75%,0.184503,-0.155204,0.097648,0.401801,0.408105,0.048418,0.353423,0.329395,0.290274,0.518959,...,0.011136,0.326888,-0.075710,0.100457,0.097969,0.085784,0.239068,0.314840,5.000000,3.000000
max,0.768264,0.358668,0.768941,1.002226,1.033958,0.625060,0.908412,0.928597,0.986870,0.976727,...,0.607897,0.791306,0.447696,0.735829,0.625892,0.635328,0.880358,0.902000,9.000000,4.000000


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.308755,-0.604654,-0.333596,0.642854,0.412059,-0.058123,-0.025220,0.481470,0.087914,0.404245,...,0.064545,-0.313434,-0.266470,-0.204130,-0.641291,0.074667,-0.046882,-0.319862,18,0
1,0.201447,-0.425371,-0.107853,0.624540,0.102533,0.131433,0.051403,0.413653,-0.127133,0.437723,...,-0.036670,-0.394913,-0.502934,-0.164598,-0.590477,-0.192863,0.140848,-0.206642,1,2
2,0.354234,-0.570777,-0.389344,0.655677,0.412454,-0.049876,-0.023400,0.413324,0.167381,0.470285,...,0.050814,-0.278357,-0.251854,-0.293655,-0.652855,-0.059429,0.021507,-0.305793,18,2
3,0.335366,-0.485172,-0.363554,0.514423,0.295635,-0.073399,0.011293,0.386332,0.164537,0.435798,...,0.012093,-0.371701,-0.138494,-0.236379,-0.441365,0.021521,0.022058,-0.372443,18,0
4,0.270679,-0.490852,-0.421678,0.594717,0.314612,-0.049140,-0.120602,0.469324,0.251018,0.405691,...,0.092495,-0.306062,-0.252897,-0.318579,-0.444948,-0.019090,-0.063598,-0.319303,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.204827,-0.401470,-0.159424,0.443596,0.309543,-0.004324,-0.096213,0.350919,0.122673,0.294656,...,-0.228286,-0.174498,-0.252752,-0.328478,-0.236182,-0.136126,0.232410,-0.202160,1,0
13180,-0.133547,-0.435242,-0.409446,-0.026667,0.504479,0.132794,0.060029,0.066413,0.332958,0.585000,...,-0.252667,0.267256,-0.166055,0.090055,0.127973,-0.672717,0.023247,0.371141,8,3
13181,-0.119726,-0.494987,-0.162720,-0.116067,0.415637,0.024566,-0.056626,-0.139355,0.443513,0.637375,...,-0.091951,0.305129,-0.097508,-0.124672,0.016707,-0.746881,0.076919,0.359549,8,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,0.078469,-0.319423,-0.041411,0.266455,0.277935,-0.101854,0.206460,0.207270,0.142193,0.402351,...,-0.086056,0.194121,-0.202356,-0.027482,-0.056685,-0.043385,0.077474,0.135018,8.206311,2.190382
std,0.196850,0.214147,0.219533,0.216230,0.192262,0.233308,0.214629,0.201992,0.213425,0.180070,...,0.181310,0.200109,0.212968,0.189137,0.220631,0.185078,0.206678,0.224756,5.419074,1.315339
min,-0.630406,-1.114412,-0.701828,-0.504687,-0.395698,-0.781871,-0.538038,-0.645521,-0.638057,-0.200081,...,-0.766940,-0.658206,-0.905668,-0.953552,-0.762533,-0.857845,-0.572642,-0.570193,0.000000,0.000000
25%,-0.043592,-0.476212,-0.189559,0.120394,0.150044,-0.279310,0.055734,0.072938,-0.017459,0.278681,...,-0.202875,0.059521,-0.354182,-0.146457,-0.217738,-0.160728,-0.066485,-0.027820,3.000000,1.000000
50%,0.079644,-0.321109,-0.051516,0.274493,0.278779,-0.109917,0.209683,0.207085,0.123448,0.405490,...,-0.081668,0.195941,-0.196201,-0.022341,-0.058541,-0.044433,0.070032,0.138995,8.000000,3.000000
75%,0.208020,-0.165185,0.093154,0.416828,0.409609,0.065416,0.360217,0.338072,0.289378,0.526677,...,0.033534,0.332157,-0.047498,0.099320,0.099136,0.079930,0.214277,0.295389,12.000000,3.000000
max,0.727791,0.376496,0.790438,1.046460,0.952575,0.669351,0.967132,0.918332,0.944611,1.128578,...,0.646149,0.815432,0.457177,0.689106,0.661031,0.718023,0.844917,0.859411,19.000000,4.000000


# Community Detection

In [299]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.44116900858585945
K = 5
NMI:  0.6498711817255415
K = 10
NMI:  0.5145889049755021
K = 20
NMI:  0.31958800184803393


In [300]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.026228756671796718
  i1 conductance: 0.026228756671796718
=>conductance: 0.026228756671796718
K = 5
  i0 conductance: 0.24569508609827803
  i1 conductance: 0.056777346557424176
  i2 conductance: 0.08955412403688266
  i3 conductance: 0.010057981303987693
  i4 conductance: 0.06873344622465373
=>conductance: 0.010057981303987693
K = 10
  i0 conductance: 0.3338562091503268
  i1 conductance: 0.2817548929462761
  i2 conductance: 0.11275026343519494
  i3 conductance: 0.3367852318775326
  i4 conductance: 0.14113911601899462
  i5 conductance: 0.0462768746722601
  i6 conductance: 0.5367924528301887
  i7 conductance: 0.10341479203754653
  i8 conductance: 0.4848851269649335
  i9 conductance: 0.386449864498645
=>conductance: 0.0462768746722601
K = 20
  i0 conductance: 0.3638882967384353
  i1 conductance: 0.2976303317535545
  i2 conductance: 0.5881967213114754
  i3 conductance: 0.31226934937566353
  i4 conductance: 0.38545454545454544
  i5 conductance: 0.5820241139934235
  

# Node Classification

In [301]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    print(f"k = {k}")
    df = dfs[k]

    # train/test split
    train, test = train_test_split(df, test_size=0.1)
    feature_col = np.arange(0,128) + 1
    X_train = train.loc[:, feature_col]
    Y_train = train.label_true
    X_test = test.loc[:, feature_col]
    Y_test = test.label_true

    # fit
    clf = svm.SVC()
    clf.fit(X_train, Y_train)

    # pred
    Y_pred = clf.predict(X_test)

    # results
    micro_f1 = metrics.f1_score(Y_test, Y_pred, average='micro')
    macro_f1 = metrics.f1_score(Y_test, Y_pred, average='macro')
    print(f"  micro_f1: {micro_f1*100:.2f}")
    print(f"  macro_f1: {macro_f1*100:.2f}")

k = 2
  micro_f1: 93.03
  macro_f1: 93.03
k = 5
  micro_f1: 91.96
  macro_f1: 91.81
k = 10
  micro_f1: 92.72
  macro_f1: 92.43
k = 20
  micro_f1: 92.42
  macro_f1: 92.09
